# EDA

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [2]:
# dirs
input_dir = '../input/'
working_dir = '../working/'
output_dir = '../output/'

In [3]:
# read data
train = pd.read_pickle(os.path.join(input_dir, 'train.pkl'))
test = pd.read_pickle(os.path.join(input_dir, 'test.pkl'))
sample_submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'), dtype={'fullVisitorId': 'str'})

In [14]:
y = train['totals.transactionRevenue'].fillna(0)
X_cols = ['visitNumber', 'device.isMobile', 'geoNetwork.continent', 'totals.newVisits', 'totals.pageviews', 'totals.visits']
X = train[X_cols]
X['geoNetwork.continent'] = X['geoNetwork.continent'].astype('category')

/home/kuriyama/miniconda3/envs/kaggle_gstore_revenue/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
gbm_params = {
    'learning_rate':[0.2]
    ,'n_estimators':[i for i in range(20, 200, 20)]
#     ,'num_leaves':[100]
}

gbm = GridSearchCV(
    estimator=lgb.LGBMRegressor(random_state=0)
    ,param_grid=gbm_params
    ,cv=5
)

In [10]:
gbm.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=31, objective=None, random_state=0,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.2], 'n_estimators': [20, 40, 60, 80, 100, 120, 140, 160, 180]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [11]:
gbm = lgb.LGBMRegressor(random_state=0)
gbm.fit(X, y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=31, objective=None, random_state=0,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [12]:
pred = gbm.predict(X)
pred = pd.Series(data=pred, name='pred')

In [13]:
pred_df = pd.concat([y, pred], axis=1)
pred_df[pred_df['totals.transactionRevenue']>0].head()

,totals.transactionRevenue,pred
752,37860000.0,1.652400e+06
753,306670000.0,9.380660e+06
799,68030000.0,1.705359e+07
802,26250000.0,1.705359e+07
859,574150000.0,2.904805e+07


In [15]:
test_X = test[X_cols]
test_X['geoNetwork.continent'] = test_X['geoNetwork.continent'].astype('category')

/home/kuriyama/miniconda3/envs/kaggle_gstore_revenue/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
test_pred = gbm.predict(test_X)
test_pred_df = pd.concat([test['fullVisitorId'], pd.Series(data=test_pred, name='PredictedLogRevenue')], axis=1)

In [39]:
submission_pred_s = test_pred_df.groupby('fullVisitorId')['PredictedLogRevenue'].sum()
submission_pred_s.head()

fullVisitorId
259678714014      5.468208e+06
3038793126460     6.915351e+04
9663019041506     3.146925e+04
10438463470860    3.146925e+04
10992980461157    3.146925e+04
Name: PredictedLogRevenue, dtype: float64

In [37]:
test['fullVisitorId'].unique().shape

(650489,)

In [26]:
sample_submission.shape

(617242, 2)

In [29]:
sample_submission.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.0
1,0000049363351866189,0.0
2,0000053049821714864,0.0
3,0000059488412965267,0.0
4,0000085840370633780,0.0


In [31]:
submission_id = sample_submission['fullVisitorId']
submission_id.index = sample_submission['fullVisitorId']

In [ ]:
pd.merge(sample_submission.drop('PredictedLogRevenue'), )

In [40]:
submission_df = pd.merge(submission_id.to_frame(), submission_pred_s.to_frame(), how='left', on='fullVisitorId')

/home/kuriyama/miniconda3/envs/kaggle_gstore_revenue/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: FutureWarning: 'fullVisitorId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
test['fullVisitorId'].info()

AttributeError: 'Series' object has no attribute 'info'